# Descriptions and Details:
use LASSO to select features, building on a pre-implemented solver for LASSO
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, I will implement my own LASSO solver, using coordinate descent.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from math import log, sqrt

In [3]:
os.getcwd()

'/Users/jessiesun/Desktop/UW-Regression/week5'

In [4]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv("data_files/kc_house_data.csv", dtype = dtype_dict)

## Create some new features

In [5]:
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

## Implement Lasso using pre-built solver

In [6]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']   # 17 features

model_all = linear_model.Lasso(alpha=1e10, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=10000000000.0, normalize=True)

In [7]:
model_all.coef_

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

## Splitting data

In [8]:
testing = pd.read_csv('data_files/wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('data_files/wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('data_files/wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [9]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [10]:
training.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,sqft_living_sqrt,sqft_lot_sqrt,bedrooms_square,floors_square
0,2487200875,20141209T000000,604000.0,4.0,3.0,1960.0,5000,1.0,0,0,...,0,98136,47.5208,-122.393,1360.0,5000.0,44.271887,70.710678,16.0,1.0
1,7237550310,20140512T000000,1225000.0,4.0,4.5,5420.0,101930,1.0,0,0,...,0,98053,47.6561,-122.005,4760.0,101930.0,73.620649,319.264780,16.0,1.0


## Selection lambda via validation set approach

In [22]:
l1_penalty = np.logspace(1, 7, num=13)
RSS_list = []
for l1 in l1_penalty:
    model = linear_model.Lasso(alpha = l1, normalize = True)
    model.fit(training[all_features], training['price'])
    prediction = model.predict(validation[all_features])
    RSS = np.sum((validation['price'] - prediction)**2)
    RSS_list.append(RSS)

In [26]:
result = pd.DataFrame({'penalty': l1_penalty, 'RSS': RSS_list})
result.set_index('penalty', inplace = True)
result
bestlam = result.index[result['RSS'] == result['RSS'].min()].tolist()
bestlam

[10.0]

## Compute RSS on test data using selected parameter

In [30]:
best_model = linear_model.Lasso(alpha = bestlam, normalize = True)
best_model.fit(training[all_features], training['price'])
test_pred = best_model.predict(testing[all_features])
RSS = np.sum((testing['price'] - test_pred)**2)
RSS

98467402552698.8

## Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [ ]:
max_nonzeros = 7

###  Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`,
and implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(1, 4, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. 
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.

In [54]:
l1_penalty_values = np.logspace(1, 4, num=20)
nzeros_dict = {}
for l1 in l1_penalty_values:
    model = linear_model.Lasso(alpha = l1, normalize = True)
    model.fit(training[all_features], training['price'])
    nzeros = sum(x != 0 for x in model.coef_) + 1  # count the number of nonzero coefficients, add 1 for constant
    nzeros_dict[l1] = nzeros
nzeros_dict

{10.0: 15,
 14.38449888287663: 15,
 20.6913808111479: 15,
 29.76351441631318: 15,
 42.81332398719393: 13,
 61.58482110660264: 12,
 88.58667904100822: 11,
 127.42749857031335: 10,
 183.29807108324357: 7,
 263.6650898730358: 6,
 379.26901907322497: 6,
 545.5594781168514: 6,
 784.7599703514607: 5,
 1128.8378916846884: 3,
 1623.776739188721: 3,
 2335.7214690901214: 2,
 3359.818286283781: 1,
 4832.930238571752: 1,
 6951.927961775606: 1,
 10000.0: 1}

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` 
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` 

In [61]:
print(np.min([l1 for l1, nzeros in nzeros_dict.items() if nzeros == 6]))  #print the min penalty of which has 6 nzeros
print(np.max([l1 for l1, nzeros in nzeros_dict.items() if nzeros == 10])) #print the max penalty of which has 10 nzeros

263.6650898730358
127.42749857031335


In [63]:
l1_penalty_max = np.min([l1 for l1, nzeros in nzeros_dict.items() if nzeros == 6])
l1_penalty_min = np.max([l1 for l1, nzeros in nzeros_dict.items() if nzeros == 10])

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [122]:
l1_penalty_narrow_range = np.linspace(l1_penalty_min,l1_penalty_max,20)
nzeros_dict = {}
for l1 in l1_penalty_narrow_range:
    model = linear_model.Lasso(alpha = l1, normalize = True)
    model.fit(training[all_features], training['price'])
    nzeros = sum(x != 0 for x in model.coef_) + 1  # count the number of nonzero coefficients, add 1 for constant
    prediction = model.predict(validation[all_features])
    RSS = np.sum((validation['price']-prediction)**2)
    nzeros_dict[l1] = nzeros, RSS
nzeros_dict

{127.42749857031335: (10, 435374677102680.6),
 134.5978981125619: (10, 437009229124471.3),
 141.76829765481045: (8, 438236128386912.25),
 148.938697197059: (8, 439158937799660.0),
 156.10909673930755: (7, 440037365263316.56),
 163.2794962815561: (7, 440777489641605.25),
 170.44989582380464: (7, 441566698090139.9),
 177.6202953660532: (7, 442406413188666.2),
 184.79069490830176: (7, 443296716874315.06),
 191.96109445055032: (7, 444239780526141.6),
 199.13149399279888: (7, 445230739842614.2),
 206.3018935350474: (6, 446268896864706.3),
 213.47229307729594: (6, 447112919434640.6),
 220.6426926195445: (6, 447998187851564.94),
 227.81309216179307: (6, 448924706673255.06),
 234.98349170404163: (6, 449892475899711.0),
 242.1538912462902: (6, 450901498778123.2),
 249.32429078853872: (6, 451952426654987.06),
 256.49469033078725: (6, 453043924367599.25),
 263.6650898730358: (6, 454176669662635.25)}

In [123]:
best_lambda = [l1 for l1, nzeros in nzeros_dict.items() if nzeros[0] == 7]     # nzeros[0] returns the number of nonzero
[nzeros_dict[x] for x in best_lambda]  # return the values where key=best_lambda

In [148]:
nzeros_df = pd.DataFrame(nzeros_dict)
nzeros_df = nzeros_df.transpose()
nzeros_df.columns = ['penalty', 'RSS']
nzeros_df.head(3)

,penalty,RSS
127.427499,10.0,4.353747e+14
134.597898,10.0,4.370092e+14
141.768298,8.0,4.382361e+14


In [151]:
final_lambda = nzeros_df.index[nzeros_df['RSS'] == nzeros_df[nzeros_df['penalty'] ==7]['RSS'].min()].tolist()

In [154]:
model = linear_model.Lasso(alpha = final_lambda, normalize = True)
model.fit(training[all_features], training['price'])
print(model.coef_)
print(model.intercept_)

[-0.00000000e+00 -0.00000000e+00  1.06108903e+04  1.63380252e+02
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  5.06451687e+05  4.19600436e+04  0.00000000e+00
  1.16253554e+05  0.00000000e+00  0.00000000e+00 -2.61223488e+03
  0.00000000e+00]
4422190.279120353
